Prepare meteorological data for WOFOST calibration

In [24]:
import pandas as pd
import xarray as xr
from pyproj import Transformer
import numpy as np
import datetime
from math import cos, sin, asin, pi, radians
import geopandas as gpd

def read_data_Fossati(datadir):
    
    """ Read data from Dario Fossati for further use. 
    - Change "Harvestyear" to "year"
    """
    
    ###### Dario Fossati data
    
    # read phenology data for wheat
    data_Fossati = pd.read_csv(datadir, delimiter=';', header=0, encoding="ISO-8859-1")
    # rename
    data_Fossati = data_Fossati.rename(columns={"Harvestyear": "year"})

    return data_Fossati

def plz_to_lonlatxy(PLZ,plzdata):
   
    """ Get coordinates based on PLZ (coordinates of community centres, in future could use centre of polygon as for damage claim data!)
    Parameters
    -----------
        PLZ: int
                PLZ to transform in lat lon
    Returns
    ----------
        lat: latitude
        lon: longitude
        x: x-coordinate (LV95)
        y: y-coordinate (LV95)
        
        """
        
    #read plzdata
    plz = pd.read_csv(plzdata,delimiter=',',header=0)

    #get EPSG2056 coords
    X=plz.loc[plz['PLZ']==PLZ]['X'].values[0]
    Y=plz.loc[plz['PLZ']==PLZ]['Y'].values[0]
    
    #change projection to lat lon   
    transformer = Transformer.from_crs("epsg:2056", "epsg:4326",always_xy=True)
    lon,lat=transformer.transform(X,Y)
    
    return lon,lat,X,Y


def get_MCH_data_lat_lon(startyear,endyear,variables,datadir,lat=None,lon=None,X=None,Y=None):

    """ Prepare Meteo Swiss data for WOFOST runs or run of phenology model
    
    Parameters
    ----------
        startyear: int
            startyear
        endyear: int
            endyear
        variables: list of strings
            list with variables to load
        lat: float
            latitude in degrees
        lon: float
            longiude in degrees
        X: X coordinate in LV95
        Y: Y coordinate in LV95
        

    Returns
    ----------            
       xarray.Dataset with variables at requested coordinate
    """
    if X and Y:
        print('Use LV95 coordinate system....')
    elif lat and lon:
        #make transformation
        print('transform lat lon to L95 coordinate system...')
        WGS84_to_LV95 = Transformer.from_crs("epsg:4326","epsg:2056",always_xy=True)
        X,Y=WGS84_to_LV95.transform(xx=lon,yy=lat)
    else:
        raise ValueError("really not enough coordinates specified.")
         
        
    startyear = int(startyear)
    endyear = int(endyear)

    time_series_list=[]
    
    for var in variables:
   
        #get list of datafiles #1961-2022
        datafiles=[datadir+str(var)+'D_ch01r.swiss.lv95_'+str(year)+'01010000_'+str(year)+'12310000.nc' for year in range(startyear,endyear+1)]
        #read datafiles
        print('reading data from {}...'.format(datadir))
        data=xr.open_mfdataset(datafiles,concat_dim = 'time',combine='nested',coords = 'minimal')
        
        #get data at lat lon
        time_series_list.append(data.sel(E=X,N=Y,method='nearest'))

    data_out=xr.merge(time_series_list)      

    return data_out

First, define directories of phenological data, and data linking PLZ and coordinates, start and end year, variables (here Tmin and Tmax) and directory to store the data

In [26]:
#produce MCH Data (comment if already done so)
#read plzdata
datadir_PLZ = 'C:/Users/F80840370/projects/scClim/wofost/PhenoSwiss/calib_data/PLZ_coordinates.txt'
datadir_Phenology = 'C:/Users/F80840370/projects/scClim/wofost/PhenoSwiss/calib_data/Phenologie_Fossati_put_together_00_18.csv'
datadir_MCH='O:/Data-Raw/27_Natural_Resources-RE/99_Meteo_Public/MeteoSwiss_netCDF/__griddedData/lv95/'
OUTDIR='C:/Users/F80840370/projects/scClim/climada/wofost/PhenoSwiss/calib_data/TminTmax_PLZ/'

startyear=1999
endyear=2021
variables=['Tmin','Tmax']
filename_base='TminTmax_daily_{}_{}'.format(startyear,endyear)

Then create the daily time series for each PLZ in the phenological dataset and store it - takes some time

In [27]:
all_pheno_data=read_data_Fossati(datadir_Phenology)
for plz in list(set(all_pheno_data['PLZ'])):
      lon,lat,X,Y=plz_to_lonlatxy(plz,plzdata)
      filename=filename_base+'_{}'.format(plz)
      data_mch=get_MCH_data_lat_lon(startyear, endyear, variables, datadir_MCH, X=X,Y=Y)
      filename='TminTmax_daily_{}_{}_{}'.format(startyear,endyear,plz)
      data_mch.to_netcdf(OUTDIR+filename+'.nc')
      data_df=data_mch.to_dataframe()
      data_df.to_excel(OUTDIR+filename+'.xlsx')

Use LV95 coordinate system....
reading data from O:/Data-Raw/27_Natural_Resources-RE/99_Meteo_Public/MeteoSwiss_netCDF/__griddedData/lv95/...


Voilá, you are ready to calibrate.